# pandapower Optimal Power Flow with piecewise linear costs
This is a copy of the OPF basic tutorial using piecewise linear costs instead of polynomial costs. Everything else is exactly the same. If you don't know the OPF basic tutorial, you might look into it first before using this one.

## Example Network

We use the following four bus example network for this tutorial:

<img src="pics/example_opf.png" width="50%">

We first create this network in pandapower:

In [27]:
import pandapower as pp
import numpy as np
net = pp.create_empty_network()

#create buses
bus1 = pp.create_bus(net, vn_kv=220.)
bus2 = pp.create_bus(net, vn_kv=110.)
bus3 = pp.create_bus(net, vn_kv=110.)
bus4 = pp.create_bus(net, vn_kv=110.)

#create 220/110 kV transformer
pp.create_transformer(net, bus1, bus2, std_type="100 MVA 220/110 kV")

#create 110 kV lines
pp.create_line(net, bus2, bus3, length_km=70., std_type='149-AL1/24-ST1A 110.0')
pp.create_line(net, bus3, bus4, length_km=50., std_type='149-AL1/24-ST1A 110.0')
pp.create_line(net, bus4, bus2, length_km=40., std_type='149-AL1/24-ST1A 110.0')

#create loads
pp.create_load(net, bus2, p_kw=60e3, controllable = False)
pp.create_load(net, bus3, p_kw=70e3, controllable = False)
pp.create_load(net, bus4, p_kw=10e3, controllable = False)

#create generators
eg = pp.create_ext_grid(net, bus1, max_p_kw= 1e9, min_p_kw = -1e9)
g0 = pp.create_gen(net, bus3, p_kw=-80*1e3, min_p_kw=-80e3, max_p_kw=0,vm_pu=1.01, controllable=True)
g1 = pp.create_gen(net, bus4, p_kw=-100*1e3, min_p_kw=-100e3, max_p_kw=0, vm_pu=1.01, controllable=True)

## Loss Minimization

We specify the same costs for the power at the external grid and all generators to minimize the overall power feed in. This equals an overall loss minimization. In the polynomial definition they represented a straight with a slope of 1. to realize this as a PWL cost, we simply make one point at the minimum power and one point at zero. For the external grid, we keep the polynomial costs. The cost types can be combined. Please note: All PWL costs should have the same number of data points!

In [28]:
costeg = pp.create_polynomial_cost(net, 0, 'ext_grid', np.array([1, 0]))
costgen1 = pp.create_piecewise_linear_cost(net, 0, 'gen', np.array([[-80*1e3, -80*1e3],[0, 0]]))
costgen2 = pp.create_piecewise_linear_cost(net, 1, 'gen', np.array([[-100*1e3, -100*1e3], [0,0]]))

We run an OPF:

In [29]:
pp.runopp(net, verbose=True)

hp.pandapower.run - INFO: These missing columns in ext_grid are considered in OPF as +- 1000 TW.: ['min_q_kvar' 'max_q_kvar']
hp.pandapower.run - INFO: These elements have missing power constraint values, which are considered in OPF as +- 1000 TW: ['gen']
hp.pandapower.run - INFO: min_vm_pu is missing in bus table. In OPF these limits are considered as 0.0 pu.
hp.pandapower.run - INFO: max_vm_pu is missing in bus table. In OPF these limits are considered as 2.0 pu.


PYPOWER Version 5.0.0, 29-May-2015 -- AC Optimal Power Flow
Python Interior Point Solver - PIPS, Version 1.0, 07-Feb-2011
Converged!


let's check the results:

In [30]:
net.res_ext_grid

,p_kw,q_kvar
0,-56530.125753,-1974.470643


In [31]:
net.res_gen

,p_kw,q_kvar,va_degree,vm_pu
0,-71313.391997,1969.654771,-3.712810,1.000009
1,-12299.771985,1451.159898,-3.712777,1.000010


Since all costs were specified the same, the OPF minimizes overall power generation, which is equal to a loss minimization in the network. The loads at buses 3 and 4 are supplied by generators at the same bus, the load at Bus 2 is provided by a combination of the other generators so that the power transmission leads to minimal losses.

## Individual Generator Costs

Let's now assign individual costs to each generator.

We assign a cost of 10 ct/kW for the external grid, 15 ct/kw for the generator g0 and 12 ct/kw for generator g1. We multiply the function value of the PWL costs with the price:

In [32]:
net.polynomial_cost.c.at[costeg] = np.array([[0.1, 0]])
net.piecewise_linear_cost.f.at[costgen1] = np.array([[0.15 * -80*1e3, 0]])
net.piecewise_linear_cost.f.at[costgen2] = np.array([[0.12 * -100*1e3, 0]])

And now run an OPF:

In [33]:
pp.runopp(net, verbose=True)

hp.pandapower.run - INFO: These missing columns in ext_grid are considered in OPF as +- 1000 TW.: ['min_q_kvar' 'max_q_kvar']
hp.pandapower.run - INFO: These elements have missing power constraint values, which are considered in OPF as +- 1000 TW: ['gen']
hp.pandapower.run - INFO: min_vm_pu is missing in bus table. In OPF these limits are considered as 0.0 pu.
hp.pandapower.run - INFO: max_vm_pu is missing in bus table. In OPF these limits are considered as 2.0 pu.


PYPOWER Version 5.0.0, 29-May-2015 -- AC Optimal Power Flow
Python Interior Point Solver - PIPS, Version 1.0, 07-Feb-2011
Converged!


We can see that all active power is provided by the external grid. This makes sense, because the external grid has the lowest cost of all generators and we did not define any constraints.

The dispatch costs are given in net.res_cost:

In [34]:
net.res_cost

14455.950328285486

Which is exactly the same result as in the Polynomial cost tutorial!